In [1]:
# 2025/7/11
# zhangzhong
# https://huggingface.co/docs/transformers/tokenizer_summary

In [ ]:
# tokenizing a text is splitting it into words or subwords,
#  which then are converted to ids through a look-up table

#  Byte-Pair Encoding (BPE), WordPiece, and SentencePiece,

In [ ]:
# Word Piece
# Space and punctuation tokenization and rule-based tokenization are both examples of word tokenization
# which is loosely defined as splitting sentences into words
# pros: While it’s the most intuitive way to split texts into smaller chunks
# cons: usually generates a very big vocabulary (the set of all unique words and tokens used), ex. Transformer XL: vocab size: 267,735
#.      Such a big vocabulary size forces the model to have an enormous embedding matrix as the input and output layer, which causes both an increased memory and time complexity.
str = "Don't you love 🤗 Transformers? We sure do."
tokens = ["Do", "n't", "you", "love", "🤗", "Transformers", "?", "We", "sure", "do", "."]

# Character Piece
# While character tokenization is very simple and would greatly reduce memory and time complexity it makes it much harder for the model to learn meaningful input representations.

In [ ]:
# subword tokenization
# hybrid between word-level and character-level tokenization

# Subword tokenization algorithms rely on the principle that 
# frequently used words should not be split into smaller subwords, 
# but rare words should be decomposed into meaningful subwords.

# annoyingly -> annoying, ly
# This is especially useful in agglutinative languages(黏着语系) such as Turkish, where you can form (almost) arbitrarily long complex words by stringing together subwords.

# Note that all of those tokenization algorithms rely on some form of training which is usually done on the corpus the corresponding model will be trained on.

# In addition, subword tokenization enables the model to process words it has [never seen before], by decomposing them into known subwords
# some example
str = "I have a new GPU!"
# 1. lowercase the text
# 2. for unseen word gpu, split to gp and u
# # "##" means that the rest of the token should be attached to the previous one, without space (for decoding or reversal of the tokenization).
tokens = ["i", "have", "a", "new", "gp", "##u", "!"]

In [ ]:
# BPE, Byte-Pair Encoding
# 这一段的解释非常精彩，一下子就懂了 https://huggingface.co/docs/transformers/tokenizer_summary#byte-pair-encoding-bpe

# 1. BPE relies on a pre-tokenizer that splits the training data into words.
#   - space tokenization, GPT2, RoBERTa
#.  - rule-based tokenization: Moses, SpaCy, ftfy
# after pre-tokenization, a set of unique words has been created, and the frequency of each word is counted.

# 2. merge 
#   - use all the symbols as a base vocabulary
#.  - learns merge rules to form a new symbol from two symbols of the base vocabulary
#.  - until the vocabulary has attain the predefined desired vocabulary size


In [ ]:
# GPT2: Byte-level BPE, OpenAI 就用这个 tiktoken
# The base vocabulary of GPT-2 consists of:
# 	•	All 256 possible byte values (from 0x00 to 0xFF) — this is known as the byte-level encoding, and it’s the core idea behind GPT-2’s tokenizer.
# 	•	These 256 bytes represent every possible character in any UTF-8 encoded text, including letters, punctuation, emojis, symbols, etc., ensuring complete coverage and eliminating the need for an <unk> token.

In [ ]:
# Word Piece
# https://huggingface.co/docs/transformers/tokenizer_summary#wordpiece

# Intuitively, WordPiece is slightly different to BPE in that it evaluates what it loses by merging two symbols to ensure it’s worth it.


In [ ]:
# Sentence Piece
# https://huggingface.co/docs/transformers/tokenizer_summary#sentencepiece

# BPE and WordPiece have the same problem:
# It is assumed that the input text uses spaces to separate words, but like chinese, which do not.
# it has two solution:
#  1. use a specific pre-tokenizer for chinese, such as XLM
#. 2. SentencePiece, treats the input as a raw input stream, thus including the space in the set of characters to use.
#.    It then uses the BPE or unigram algorithm to construct the appropriate vocabulary.
#.    其实就是把空格加入到字典中呗。在字典中用下划线 _ 来表示空格。

# example:
str = "Don't you love 🤗 Transformers? We sure do."
tokens = ["▁Don", "'", "t", "▁you", "▁love", "▁", "🤗", "▁", "Transform", "ers", "?", "▁We", "▁sure", "▁do", "."]

BPE（Byte Pair Encoding）算法在用于 NLP 中（尤其是 GPT 系列模型）时的 tokenize 过程，可以分为两个阶段：训练阶段和应用阶段（tokenization）。下面我们重点讲应用阶段：给定一个句子，BPE 是如何把它转成 token 的。

⸻

🌟 简化版解释：BPE Tokenize 的流程

输入示例：

文本： "lower"

假设已经训练好的 BPE 合并规则如下：

1. l o w e r
2. lo w e r
3. low e r
4. low er

最终得到的 token 是：

["low", "er"]


⸻

🧠 GPT 中的 Byte-Level BPE Tokenization 详细流程：

以 GPT-2 使用的 BPE 为例，tokenize 的步骤如下：

⸻

✅ 步骤 1：文本转为字节（byte）

GPT-2 先将输入文本按 UTF-8 编码为字节流（不是字符！）。

Input: "hello 😊"
→ bytes: [104, 101, 108, 108, 111, 32, 240, 159, 152, 138]

然后每个字节对应一个基础 token，构成初始 token 列表（共 256 个基础 token）。

⸻

✅ 步骤 2：将字节序列映射为基础 token（base vocabulary）

例如：

Input: "hello"
→ bytes: ['h', 'e', 'l', 'l', 'o']
→ tokens: ["h", "e", "l", "l", "o"]


⸻

✅ 步骤 3：使用 BPE Merge 规则进行合并

BPE Merge Rules 是按照训练时统计频率学到的一系列「字符对合并规则」，如：

("l", "l") → "ll"
("e", "ll") → "ell"
("h", "ell") → "hell"
("hell", "o") → "hello"

每次找到词表中已有的最高优先级的 pair 合并成新 token，直到无法继续合并。

最终输出 token 列表：

["hello", "😊"]


⸻

✅ 步骤 4：每个 token 映射为整数 ID

GPT 模型不直接使用文本 token，而是用它们在词表中的位置（整数索引）表示：

["hello", "😊"] → [31373, 50256]  # 示例 ID


⸻

🧪 实际例子（使用 tiktoken + GPT2）

import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("hello world!")
print(tokens)
# 输出: [31373, 995]  → 分别是 "hello" 和 " world!"


⸻

✅ BPE 的优势

优点	说明
🔤 无 <unk>	所有字节都在基础词表中，不存在无法处理的字符
🔄 可逆	每个 token 可以反向恢复原文本
📦 词表小	只需要 256 个基础 token + 若干合并 token（GPT-2 是 50257）
📈 支持词干共享	不同词有相同词根时可以复用（例如 play, playing, played）


⸻

🧩 总结一句话：

BPE Tokenization 是从字节出发，通过反复合并频繁出现的字符对，最终将文本拆解为模型能识别的 token ID 的过程。

⸻

需要我帮你可视化某个具体句子的 BPE tokenization 步骤吗？

In [ ]:
# 合并的伪代码
# 给定一个token序列，这是一句话展开成byte的序列: list[str]
# 然后顺序遍历这个列表，每两个相邻的符号，在merge table中找到相应的合并规则，优先合并靠前的
# 重复这个过程，直到不能合并，剩下的就是token过后的

# Byte-level BPE Tokenization

非常好，那我们现在以一个具体例子来详细演示一下 GPT-2 所使用的 Byte-level BPE 分词（tokenization）全过程，从文本 → 字节 → 基础 token → BPE 合并 → token id。

⸻

🌟 示例文本：

我们用包含中文、emoji、生僻字的字符串：

"你好，GPT-2 👋 𩸽"


⸻

✅ Step 1：文本转为 UTF-8 字节序列

Python 中我们可以用 .encode("utf-8") 来查看底层字节：

text = "你好，GPT-2 👋 𩸽"
utf8_bytes = text.encode("utf-8")
print(list(utf8_bytes))

输出的字节（十进制）为：

['你' → e4 bd a0]  
['好' → e5 a5 bd]  
['，' → ef bc 8c]  
['G' → 47]  
['P' → 50]  
['T' → 54]  
['-' → 2d]  
['2' → 32]  
[' ' → 20]  
['👋' → f0 9f 91 8b]  
[' ' → 20]  
['𩸽' → f0 a9 b8 bd]

合起来是：

[228, 189, 160, 229, 165, 189, 239, 188, 140, 71, 80, 84, 45, 50, 32, 240, 159, 145, 139, 32, 240, 169, 184, 189]

每一个数字都在 GPT-2 的 base vocabulary（256个byte）中。

⸻

✅ Step 2：将字节序列转为基础 token（初始 tokens）

此时，我们将每个 byte 单独作为一个 token，等价于：

initial_tokens = [chr(b) for b in utf8_bytes]

结果类似：

['\xe4', '\xbd', '\xa0',  # '你'
 '\xe5', '\xa5', '\xbd',  # '好'
 '\xef', '\xbc', '\x8c',  # '，'
 'G', 'P', 'T', '-', '2', ' ',
 '\xf0', '\x9f', '\x91', '\x8b',  # 👋
 ' ',
 '\xf0', '\xa9', '\xb8', '\xbd']  # 𩸽


⸻

✅ Step 3：应用 BPE 合并规则（Byte Pair Merging）

GPT-2 在预训练时生成了一个排序好的「合并规则列表」（50,000个），用于按频率从这些字符对中找到最优合并。

每次合并规则类似这样：

("e", "l") → "el"
("el", "l") → "ell"
("ell", "o") → "ello"

在我们的例子中，tokenizer 会一步步合并这些字节对成更长的 token（比如：“你”、“好”、“GPT”、“👋”、“𩸽”）。

这些合并规则由 GPT-2 在训练时从海量文本中学习得出 —— 高频组合会被优先合并。

⸻

✅ Step 4：映射到 token id

最终合并后的每个 token（无论是单个字符、组合词，还是 emoji）都会有一个唯一的 token ID，比如：

Token	Token ID
“你”	21940
“好”	23494
“，”	232
“GPT”	12172
“ “	220
“👋”	67909
“𩸽”	86000+（可能是 byte 级组合）

你可以用如下代码打印完整 token 序列和对应 ID 👇

⸻

🔧 使用 tiktoken 实际演示代码：

import tiktoken

enc = tiktoken.get_encoding("gpt2")
text = "你好，GPT-2 👋 𩸽"

# 编码：字符串 -> token id
token_ids = enc.encode(text)
print("Token IDs:", token_ids)

# 解码：token id -> 字符串
tokens = [enc.decode([id]) for id in token_ids]
print("Tokens:", tokens)

示例输出（可能略有不同）：

Token IDs: [21940, 23494, 232, 12172, 12, 220, 67909, 220, 11359, 262]
Tokens: ['你', '好', '，', 'GPT', '-', ' ', '👋', ' ', '𩸽', '']


⸻

🎯 总结

步骤	说明
① 文本 → 字节	每个字符变成 1~4 个字节（UTF-8）
② 字节 → 初始 token	每个字节映射为基础词表 token（共 256 个）
③ BPE 合并	使用训练好的 BPE 规则合并频繁 byte 序列为 token
④ token → ID	每个最终 token 映射为模型词表中的整数 ID


⸻

这个机制让 GPT-2 能处理世界上几乎所有语言、表情、符号，而不用担心无法识别的字符。

⸻

想让我用 tiktoken 帮你解析一段具体文本（比如 emoji、代码、罕见汉字），可以直接发我句子，我来分步演示给你看 😊

# Byte-level BPE 训练过程中，中文subword的问题
这是一个非常深入而关键的问题，尤其在中文和多语言建模中经常被提到。

简要回答：
Byte-Level BPE 的确会在中文等 Unicode 文本中产生“无意义的 subword”（例如把一个汉字拆成两部分），但 GPT-2 不会避免它，而是依赖模型去“学会”这种 token 组合的语言意义。对于中文来说，这种 subword 不总是语义合理，但在大规模训练中，模型仍能学出有效表示。

⸻

🔍 为什么会出现“无意义 subword”？

举个例子：

假设语料中有两个汉字：“你好”、“你们”。

在 UTF-8 编码下：
	•	"你" → e4 bd a0 （3 bytes）
	•	"好" → e5 a5 bd
	•	"们" → e4 bb ac

一开始每个字节是单独的 token，然后 BPE 会基于频率合并字节：

可能的合并结果是：
	•	e4 + bd → e4bd
	•	e4bd + a0 → 你
	•	但也可能只合并成 e4bd，没有继续合成完整的“你”

这样就可能出现中间态 token：

["e4bd", "a0"] ← 拆开的“你”
["e4", "bda0"] ← 异常组合

这在语义上是无意义的 subword，但 tokenizer 并不会主动避免这种情况。

⸻

✅ GPT 的策略：不避免，而是容忍

GPT 模型采取的做法是：

策略	说明
Byte-Level BPE	所有文本以 byte 拆分，保证每种字符都能被编码（避免 <unk>）
合并按频率	不考虑语义，完全按字节对出现频率来学习 subword
结果容忍冗余	即使某些合并的 token 无意义，模型可以学习如何解释它们
训练自动修正语义	模型在大规模语料中会自己学会哪些组合更有意义


⸻

🧠 这样做的优点
	•	✔ 跨语言统一处理，不需要为中文专门造词表。
	•	✔ 无OOV问题，任何字符组合都可以编码。
	•	✔ 支持稀有字符（如 emoji、罕见汉字、生僻字）。
	•	✔ 处理拼写错误、变体能力强。

⸻

❌ 缺点也明显：
	•	❌ 中文 subword 不总是语义合理，如把一个汉字拆成多个 token。
	•	❌ 语义上冗余、不清晰，例如把“语”拆成“讠”和“吾”的 byte。

⸻

🔄 那有没有更好的办法？

是的，有：

方法	描述
WordPiece（如BERT）	中文按字划分（每个汉字是一个 token），不会拆字节
SentencePiece (Unigram)	更智能地选择 subword，适合中文等非空格分词语言
专门为中文设计的 tokenizer	如 jieba、thulac、LTP 分词配合词粒度训练


⸻

💡 实际应用建议：

场景	推荐 Tokenizer
英文、代码、emoji 丰富	GPT2-style Byte-Level BPE（tiktoken）
中文、日文、韩文等	SentencePiece (Unigram) 或 中文分词+BPE
多语言混合文本	XLM-R, mBERT, SentencePiece 更合适


⸻

✅ 总结一句话：

GPT-2 的 Byte-Level BPE 不会主动避免中文中“无意义”的 subword，而是通过海量训练语料让模型自动学习有意义的组合。这种设计牺牲了一些语言学合理性，换来了无损、通用、跨语言的 tokenization 能力。

⸻

如果你希望我演示在中文语料上如何训练 BPE（比如使用 tokenizers 或 sentencepiece），我可以写一个完整流程给你看，包括如何让 tokenizer 更合理地处理中文。是否需要？